# CW-Husky Sequenced Triggers

Another new Husky feature is *sequenced triggers*. This is a very powerful feature: it allows the capture (or glitch) to be triggered after a sequence of *multiple* trigger events.

For example, you could specify a trigger sequence such that Husky triggers when it gets a UART pattern match trigger followed by a SAD trigger.

Additionally, you can (optionally) specify that the SAD trigger must follow the UART trigger by no more than $x$ clock cycles and no less than *y* clock cycles.

This notebook explains how to set up the trigger sequencer.

For a "real-world" application of sequenced triggers, have a look at the [sca205](https://github.com/newaetech/chipwhisperer-jupyter/tree/master/courses/sca205) series of notebooks which culminate by showing in 
[part 3](../../courses/sca205/uecc_part3_trace_sad.ipynb) the advantages of sequencing a jittery trace trigger with a stable SAD trigger to greatly simplify an ECC attack.

But first, let's start with the basics:

In [ ]:
SCOPE="OPENADC"
PLATFORM = 'CW308_SAM4S'
#PLATFORM = 'CW308_STM32F3'

In [ ]:
import chipwhisperer as cw

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

By default, the trigger sequencer is disabled. If you already know how to set up triggers on Husky and you're not interested in using the sequencer, then you don't need to change anything.

In [ ]:
scope.trigger

When you turn on the sequencer, several more properties appear:

In [ ]:
scope.trigger.sequencer_enabled = True
print(scope.trigger)

First, we see that Husky supports `scope.trigger.max_sequenced_triggers = 2` sequenced triggers (4 in the case of Husky Plus). This isn't something you can modify; it's a hardware limitation.

Next, you'll notice that `scope.trigger.module` and `scope.trigger.triggers` now become "`SequenceTriggerList()`" objects.

`SequenceTriggerList` is a convenience class that allows us to set or get the individual trigger modules and their triggers (if you've used Husky's multiple glitches, the same approach is used there).

The `scope.trigger` object also prints an easier-to-parse version of the trigger parameters under "sequence trigger \#0" and "sequence trigger \#1" (and \#2, \#3 in the case of Husky Plus).

We see that by default, all are set to the basic trigger module on tio4 (not very interesting).

We'll get to the `window_start` and `window_end` properties later.

You can change trigger settings by indexing the `scope.trigger.module` and `scope.trigger.triggers` properties.

Let's change the second trigger to the UART trigger module, using the tio2 line:

In [ ]:
scope.trigger.module[1] = 'UART'
scope.trigger.triggers[1] = 'tio2'

In [ ]:
scope.trigger

Husky has several trigger modules available:
- basic
- ADC
- SAD
- UART
- edge_counter
- trace

**Only the "basic" trigger module can be used more than once in a trigger sequence.**

We previously set the second trigger module to UART; if you try to set the first trigger module to UART also, you'll get an error:

In [ ]:
scope.trigger.module[0] = 'UART'

The UART and trace trigger modules share the same hardware, so they also cannot be both active:

In [ ]:
scope.trigger.module[0] = 'trace'

# First trigger sequence: GPIO4 + UART

Let's return to our legal trigger sequence: basic on tio4 followed by UART on tio1.

With the stock `simpleserial-trace` firmware (the same used in the [02 - Husky Triggers.ipynb](02%20-%20Husky%20Triggers.ipynb) notebook), we could have the trigger sequence initiated by the target's `trigger_high()`, and completed when the target sends its response on tio1.

Let's first program the correct firmware:

In [ ]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/simpleserial-trace/simpleserial-trace-{}.hex".format(PLATFORM))
reset_target(scope)

Let's first do a non-sequenced, tio4-triggered capture:

In [ ]:
scope.trigger.sequencer_enabled = False
scope.gain.db = 22
scope.adc.presamples = 0
scope.adc.samples = 50000
basictrace = cw.capture_trace(scope, target, plaintext=bytearray(range(16)), key=bytearray(16), always_send_key=True)

In [ ]:
target.simpleserial_last_read

You should see that the target responded with `r7ACA...`, so we'll set our UART trigger to trigger on that (see [02 - Husky Triggers.ipynb](02%20-%20Husky%20Triggers.ipynb) for more on UART trigger setup):

In [ ]:
scope.trigger.sequencer_enabled = True
scope.trigger.num_triggers = 2

scope.trigger.module[0] = 'basic'
scope.trigger.triggers[0] = 'tio4'

scope.trigger.module[1] = 'UART'
scope.trigger.triggers[1] = 'tio1'

In [ ]:
scope.UARTTrigger.enabled = True
scope.UARTTrigger.baud = 38400
scope.UARTTrigger.set_pattern_match(0, 'r7')
scope.UARTTrigger.trigger_source = 0
scope.UARTTrigger.rules_enabled = [0]

Let's take note of how long the operation took on our previous capture; we'll use this information later:

In [ ]:
cycles = scope.adc.trig_count
print('The target operation took %d cycles.' % cycles)

Since the capture will now trigger on the target response, let's set `presamples` to the maximum, so that we can still catch part of the target operation:

In [ ]:
scope.adc.presamples = 32767

In [ ]:
seqtrace = cw.capture_trace(scope, target, bytearray(range(16)), bytearray(16))

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
output_notebook(INLINE)
s = figure(plot_width=1800)
s.line(range(scope.adc.samples), basictrace.wave, line_color='blue')
s.line(range(scope.adc.samples), seqtrace.wave - 0.5, line_color='red')
show(s)

You should see that the red (sequence-triggered) trace is a delayed version of the blue (basic-triggered) trace.

The `scope.trigger.window_start` and `scope.trigger.window_end` parameters allow specifying when the second trigger is allowed to occur, relative to the first trigger. When these values are 0, there is no limit: the second trigger can come at any time (following the first trigger).

If `scope.trigger.window_start` is set to a non-zero value, then a second trigger which comes earlier than `window_start` clock cycles (of the ADC sampling clock) will be ignored. Having a too-early trigger is ok: the trigger sequencer will remain on the lookout for a legal second trigger. 

Similarly, if `scope.trigger.window_end` is set to a non-zero value, then a second trigger which comes after `window_end` clock cycles (of the ADC sampling clock) will be ignored. At this point the sequence can be considered "dead", however a re-occurence of the *first* trigger can still lead to a successful completion of the sequence (until the capture times out).

The window parameters can be particularly useful, for example, when the second trigger is a SAD pattern which could fire several times during the target operation, but you're only interested in SAD triggers that are expected to occur in some relatively narrow window following another trigger. This can in turn make it easier to tune the SAD trigger (i.e. if SAD is only allowed to trigger in a narrow window, then one does not need to be as careful when choosing the SAD reference and threshold).

When using `window_start`, it's important to understand that this specifies the earliest time that the trigger event can *start*, which for all trigger modules except for the basic trigger, is earlier than the time when the trigger would actually fire (e.g. with the UART trigger module, it's the *start* of the matching UART pattern must come after `window_start` -- not the end of the UART pattern).

Finally, a note on indexing: window parameters are relative to the *previous* trigger. So, the first trigger doesn't have any window parameters (there is nothing for it to be relative to). The first set of window parameters are for the second trigger, and they are relative to the first trigger. In general, `scope.trigger.window_start[i]` and `scope.trigger.window_end[i]` are the window parameters for `scope.trigger.module[i+1]`. (Since Husky supports only 2 sequenced triggers, it has only one set of window parameters; however Husky Plus supports 4 sequenced triggers, so it has three sets of window parameters.)

Let's illustrate how the window works using our existing example. First let's set up a window which prevents the trigger sequence from completing, by making the window start well after the target operation is done:

In [ ]:
scope.trigger.window_start[0] = cycles*2
scope.trigger.window_end[0] = 0
failtrace = cw.capture_trace(scope, target, bytearray(range(16)), bytearray(16))

You should get a "Timeout happened during capture" message, because the trigger sequence didn't fire.

Now let's set the window to something that can work: we make the window start shortly before the target operation completes.

In [ ]:
scope.trigger.window_start[0] = int(cycles*0.9)
scope.trigger.window_end[0] = 0
seqtrace = cw.capture_trace(scope, target, bytearray(range(16)), bytearray(16))

This next part is completely optional, but if you have an external logic analyzer, then some of the internal trigger sequence signals can be routed to the USERIO header; this can be very helpful in understanding why a trigger sequence isn't working as intended:

In [ ]:
scope.userio.mode = 'fpga_debug'
scope.userio.fpga_mode = 13

The `scope.userio.fpga_mode` property doesn't give you any useful information, but after it's set, printing the `scope.userio` object will show you which signal is routed to each USERIO pin:

In [ ]:
scope.userio

- trigger 0/1 are the internal signals for the first and second triggers of the trigger sequence
- "too early" is a pulse when the next trigger in the trigger sequence arrives prior to its window. For most trigger modules, this cannot actually happen since the active trigger module is only turned on during its window; the exceptions are the "basic" module, and the SAD module *if* `scope.trigger.sad_always_active` is set.
- "too late" pulses if the trigger that's currently being waited for is not received by the end of its window
- "sequence trigger output" pulses when the trigger sequence is completed
- "trigger 0 window" is high when the trigger sequencer is waiting for the first trigger; it goes low when the trigger is received
- "trigger 1 window" is high when the trigger sequencer is waiting for the second trigger; it goes low when the trigger is received, or its expected window expires
- "armed_and_ready" is high when the trigger sequencer is active and ready to go

If you re-run the captures for both the "bad" and "good" window setups, you should get a good handle on how the USERIO outputs can be helpful.

***Important note**: if you connect a logic analyzer to the USERIO pins, ensure it's well grounded and use quality jumper cables. The internal trigger signals are narrow single-cycle pulses, and less-than-ideal connections can actually mess up the FPGA's proper functioning (i.e. this can cause the capture that would have worked without the logic analyzer to fail). You may need to connect several ground lines between your logic analyzer and Husky.
If you suspect this is a problem (i.e. you can't get a sequence-triggered capture to work), try unconnecting from the USERIO pins, or setting `scope.userio.mode` back to `'normal'`.*

# Second trigger sequence: UART + SAD

Let's now set up a trigger sequence which doesn't use the basic GPIO4 trigger.

The first trigger will be the UART message sent to the target on GPIO2, which is `p00010203....`:

In [ ]:
target.simpleserial_last_sent

In [ ]:
scope.trigger.module[1] = 'SAD' # specify this first to prevent error!
scope.trigger.module[0] = 'UART'
scope.trigger.triggers[0] = 'tio2'
scope.UARTTrigger.set_pattern_match(0, 'p0001020')

The second trigger will be a SAD trigger. Let's grab a suitable SAD reference from the AES operation:

In [ ]:
s = figure(plot_width=1800)
s.line(range(scope.adc.samples), basictrace.wave, line_color='blue')
show(s)

In [ ]:
# this will work well for the SAM4S target; if you're using a different target, or different firmware, make sure you use something distinctive:
scope.SAD.reference = basictrace.wave[5000:]

In [ ]:
s = figure(plot_width=1800)
s.line(range(scope.SAD.sad_reference_length), scope.SAD.reference)
show(s)

Some adjustment to the SAD threshold may be required, but this should be in the right ballpark:

In [ ]:
scope.SAD.threshold = scope.SAD.sad_reference_length * 3

Let's disable the sequence window for now:

In [ ]:
scope.trigger.window_start = 0
scope.trigger.window_end = 0

In [ ]:
seqtrace = cw.capture_trace(scope, target, bytearray(range(16)), bytearray(16))

Next let's tighten our sequence window. But first, let's make the UART trigger closer to the start of the operation. The UART pattern matcher is limited to 8 bytes, which is why our pattern was set to:

In [ ]:
scope.UARTTrigger.rules[0]['patt']

Let's instead match on the *end* of the 'p' message, to move the UART trigger as close as possible to the start of the target operation:

In [ ]:
target.simpleserial_last_sent

In [ ]:
scope.UARTTrigger.set_pattern_match(0, 'c0d0e0f\n')

Our SAD reference was taken from sample 5000 onwards. The SAD module triggers at the *end* of the SAD pattern, plus some latency defined by `scope.SAD.latency`.

So let's set a narrow window for the SAD trigger around this:

In [ ]:
trigger_expected = 5000 + scope.SAD.sad_reference_length + scope.SAD.latency

...but this is relative to the GPIO4 trigger; here, the UART trigger will actually be a fair bit earlier than the GPIO4 trigger, by about 2650 cycles, so let's account for that:

In [ ]:
scope.trigger.window_start = trigger_expected + 2650 - 500
scope.trigger.window_end = trigger_expected + 2650 + 500

In [ ]:
seqtrace = cw.capture_trace(scope, target, bytearray(range(16)), bytearray(16))

If you look at the USERIO pins with a logic analyzer, you should be able to confirm that the 2nd trigger is indeed where we expect it:

<img src='img/uart_sad_window.png' width=1600>

Our SAD reference occurs several times during the target operation, so if we move the window, the capture should still work:

In [ ]:
scope.trigger.window_end = 0
scope.trigger.window_start = trigger_expected + 2650 + 10000

In [ ]:
seqtrace_later = cw.capture_trace(scope, target, bytearray(range(16)), bytearray(16))

You can visualize this either via the USERIO signals, or by comparing the captured waveforms:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
output_notebook(INLINE)
s = figure(plot_width=1800)
s.line(range(scope.adc.samples), seqtrace.wave - 0.5, line_color='red')
s.line(range(scope.adc.samples), seqtrace_later.wave - 1, line_color='purple')
show(s)

(Remember that we have `scope.adc.presamples = 32767`; you should recognize the SAD reference pattern, and perfect alignment between the two traces, around there.)

# Husky-Plus Only: 4 Triggers

With the "regular" Husky, the maximum number of sequenced triggers is 2, but with the "Plus", you can go up to 4; let's show that by adding the basic and ADC level triggers to the mix.

The sequence will be UART, GPIO4, SAD, ADC.

The UART and SAD triggers will be unchanged; we know that GPIO4 goes high in between, so that's easy; and we'll wrap up the sequence with an easy ADC trigger.

In [ ]:
assert scope._is_husky_plus, "Sorry, only 2 triggers here :-("

In [ ]:
scope.trigger.num_triggers = 4

scope.trigger.module[1] = 'basic'
scope.trigger.triggers[1] = 'tio4'
scope.trigger.window_start[0] = 2000
scope.trigger.window_end[0] = 3000

scope.trigger.module[2] = 'SAD'
scope.trigger.window_start[1] = trigger_expected-500
scope.trigger.window_end[1] = trigger_expected+500

scope.trigger.module[3] = 'ADC'
scope.trigger.level = min(basictrace.wave)*0.9
scope.trigger.window_start[2] = 1000 # arbitrary!
scope.trigger.window_end[2] = 0

If you're probing USERIO, let's change the mode to get access to all 4 internal triggers and their windows:

In [ ]:
scope.userio.fpga_mode = 15

In [ ]:
scope.userio

In [ ]:
seqtrace_4triggers = cw.capture_trace(scope, target, bytearray(range(16)), bytearray(16))

With a logic analyzer you can visualize the sequence of triggers:

<img src='img/plus_4trigger_sequence.png' width=2000>

# Next Steps

Check out the [sca205](https://github.com/newaetech/chipwhisperer-jupyter/tree/master/courses/sca205) series of notebooks which culminate by showing in 
[part 3](../../courses/sca205/uecc_part3_trace_sad.ipynb) the advantages of sequencing a jittery trace trigger with a stable SAD trigger to greatly simplify an ECC attack!